In [ ]:
# 필요 코드 필수
!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.5 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=4eadb1e87b94eb100fa255e2559b1827e96c99dc479c15d10b948788fac6ec0a
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=f0146349b6c237a2aeb9799589e9f62a397e79f55df62f02992edb5c4090059f
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision import transforms
from PIL import Image
import os
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import random
import segmentation_models_pytorch as smp

# 모델 정의(deeplabv3)
def create_deeplabv3plus(num_classes):
    model = smp.DeepLabV3Plus(encoder_name="resnet101", classes=num_classes, encoder_weights="imagenet")
    return model

# 데이터셋 클래스 정의
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, label_dir, image_list, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = image_list

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx].replace('.jpg', '_mask.png'))
        image = Image.open(img_path).convert('RGB')
        label = Image.open(label_path).convert('L')
        if self.transform:
            image = self.transform(image)
            label = self.transform(label) * 255
        return image, label.squeeze().long()

# 데이터 분할 함수
def split_dataset(image_dir, test_ratio=0.2):
    images = os.listdir(image_dir)
    random.seed(36)
    random.shuffle(images)
    split_idx = int(len(images) * (1 - test_ratio))
    return images[:split_idx], images[split_idx:]

# 데이터 변환 정의
train_transforms = transforms.Compose([
    transforms.Resize((640, 368)),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((640, 368)),
    transforms.ToTensor(),
])

# 데이터셋 경로 설정
image_dir = '/content/drive/MyDrive/횡단보도/segmentation/images'
label_dir = '/content/drive/MyDrive/횡단보도/segmentation/labels'

# 데이터셋 분할
train_images, test_images = split_dataset(image_dir)
train_dataset = SegmentationDataset(image_dir, label_dir, train_images, transform=train_transforms)
test_dataset = SegmentationDataset(image_dir, label_dir, test_images, transform=test_transforms)

# DataLoader 설정
test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False)

# 5폴드 교차검증 설정
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

results = []

# 모델 저장 경로 설정
model_save_path = '/content/drive/MyDrive/횡단보도/segmentation/models'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
    print(f'Starting fold {fold}')
    train_subsampler = SubsetRandomSampler(train_ids)
    val_subsampler = SubsetRandomSampler(val_ids)

    train_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=train_subsampler)
    val_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=val_subsampler)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_deeplabv3plus(num_classes=4).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # 검증 세트에서 모델 평가
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        # 여기서 검증 손실을 results 리스트에 추가
        val_loss /= len(val_loader)
        results.append(val_loss)
        print(f'Fold {fold}, Epoch {epoch+1}, Training Loss: {train_loss / len(train_loader)}, Validation Loss: {val_loss}')

        # 모델 저장
        torch.save(model.state_dict(), os.path.join(model_save_path, f'model_fold{fold}_epoch{epoch+1}.pth'))

# 평균 K-Fold 검증 손실 계산 및 출력
average_val_loss = np.mean(results)
print(f'Average K-Fold Validation Loss: {average_val_loss}')


Starting fold 0


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 170M/170M [00:00<00:00, 247MB/s]


Fold 0, Epoch 1, Training Loss: 0.6669338215142488, Validation Loss: 3.024048924446106
Fold 0, Epoch 2, Training Loss: 0.29531022906303406, Validation Loss: 0.4765893593430519
Fold 0, Epoch 3, Training Loss: 0.21807843446731567, Validation Loss: 0.29813849925994873
Fold 0, Epoch 4, Training Loss: 0.18207127321511507, Validation Loss: 0.24786684662103653
Fold 0, Epoch 5, Training Loss: 0.1519895694218576, Validation Loss: 0.24102819338440895
Fold 0, Epoch 6, Training Loss: 0.13324134703725576, Validation Loss: 0.22161061316728592
Fold 0, Epoch 7, Training Loss: 0.13304656650871038, Validation Loss: 0.26869943737983704
Fold 0, Epoch 8, Training Loss: 0.1434653284959495, Validation Loss: 0.3761737272143364
Fold 0, Epoch 9, Training Loss: 0.1363366055302322, Validation Loss: 0.32023219391703606
Fold 0, Epoch 10, Training Loss: 0.10810294840484858, Validation Loss: 0.20734814926981926
Fold 0, Epoch 11, Training Loss: 0.10192775446921587, Validation Loss: 0.18118765763938427
Fold 0, Epoch 12

In [ ]:
!pip install segmentation_models_pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision import transforms
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import KFold
import random
import segmentation_models_pytorch as smp

def create_deeplabv3plus(num_classes):
    model = smp.DeepLabV3Plus(encoder_name="resnet101", classes=num_classes, encoder_weights="imagenet")
    return model

class SegmentationDataset(Dataset):
    def __init__(self, image_dir, label_dir, image_list, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = image_list

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx].replace('.jpg', '_mask.png'))
        image = Image.open(img_path).convert('RGB')
        label = Image.open(label_path).convert('L')
        if self.transform:
            image = self.transform(image)
            label = self.transform(label) * 255
        return image, label.squeeze().long()

def split_dataset(image_dir, test_ratio=0.2):
    images = os.listdir(image_dir)
    random.seed(36)
    random.shuffle(images)
    split_idx = int(len(images) * (1 - test_ratio))
    return images[:split_idx], images[split_idx:]

train_transforms = transforms.Compose([
    transforms.Resize((640, 368)),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((640, 368)),
    transforms.ToTensor(),
])

image_dir = '/content/drive/MyDrive/횡단보도/segmentation/images'
label_dir = '/content/drive/MyDrive/횡단보도/segmentation/labels'

train_images, test_images = split_dataset(image_dir)
train_dataset = SegmentationDataset(image_dir, label_dir, train_images, transform=train_transforms)
test_dataset = SegmentationDataset(image_dir, label_dir, test_images, transform=test_transforms)

test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False)

k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

results = []
model_save_path = '/content/drive/MyDrive/횡단보도/4models'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# fold 4에 대해서만 학습
selected_fold = 4
for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
    if fold == selected_fold:
        print(f'Starting fold {fold}')
        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=train_subsampler)
        val_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=val_subsampler)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = create_deeplabv3plus(num_classes=4).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        num_epochs = 100

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

            val_loss /= len(val_loader)
            results.append(val_loss)
            print(f'Fold {fold}, Epoch {epoch+1}, Training Loss: {train_loss / len(train_loader)}, Validation Loss: {val_loss}')

            torch.save(model.state_dict(), os.path.join(model_save_path, f'model_fold{fold}_epoch{epoch+1}.pth'))
        break  # fold 4에 대한 학습이 끝나면 반복문을 종료합니다.

average_val_loss = np.mean(results)
print(f'Average K-Fold Validation Loss: {average_val_loss}')


Starting fold 4
Fold 4, Epoch 1, Training Loss: 0.716089403256774, Validation Loss: 1.5590717196464539
Fold 4, Epoch 2, Training Loss: 0.3193270694464445, Validation Loss: 0.5164721980690956
Fold 4, Epoch 3, Training Loss: 0.2499748570844531, Validation Loss: 0.3399377167224884
Fold 4, Epoch 4, Training Loss: 0.20049986615777016, Validation Loss: 0.24938585236668587
Fold 4, Epoch 5, Training Loss: 0.17381377797573805, Validation Loss: 0.19990120083093643
Fold 4, Epoch 6, Training Loss: 0.1496152407489717, Validation Loss: 0.17935940995812416
Fold 4, Epoch 7, Training Loss: 0.1352172987535596, Validation Loss: 0.1607186757028103
Fold 4, Epoch 8, Training Loss: 0.1216344847343862, Validation Loss: 0.16535346768796444
Fold 4, Epoch 9, Training Loss: 0.11712434142827988, Validation Loss: 0.19657901674509048
Fold 4, Epoch 10, Training Loss: 0.12683600513264537, Validation Loss: 0.3297874331474304
Fold 4, Epoch 11, Training Loss: 0.13468732126057148, Validation Loss: 0.3628617823123932
Fold 

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision import transforms
from PIL import Image
import os
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import random
import segmentation_models_pytorch as smp


def create_deeplabv3plus(num_classes):
    model = smp.DeepLabV3Plus(encoder_name="resnet101", classes=num_classes, encoder_weights="imagenet")
    return model

# 데이터셋 클래스 정의
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, label_dir, image_list, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = image_list

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx].replace('.jpg', '_mask.png'))
        image = Image.open(img_path).convert('RGB')
        label = Image.open(label_path).convert('L')
        if self.transform:
            image = self.transform(image)
            label = self.transform(label) * 255
        return image, label.squeeze().long()

# 데이터 분할 함수
def split_dataset(image_dir, test_ratio=0.2):
    images = os.listdir(image_dir)
    random.seed(36)
    random.shuffle(images)
    split_idx = int(len(images) * (1 - test_ratio))
    return images[:split_idx], images[split_idx:]

# 데이터 변환 정의
train_transforms = transforms.Compose([
    transforms.Resize((640, 368)),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((640, 368)),
    transforms.ToTensor(),
])

# 데이터셋 경로 설정
image_dir = '/content/drive/MyDrive/횡단보도/segmentation/images'
label_dir = '/content/drive/MyDrive/횡단보도/segmentation/labels'

# 데이터셋 분할
train_images, test_images = split_dataset(image_dir)
train_dataset = SegmentationDataset(image_dir, label_dir, train_images, transform=train_transforms)
test_dataset = SegmentationDataset(image_dir, label_dir, test_images, transform=test_transforms)

# DataLoader 설정
test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False)

# 5폴드 교차검증 설정
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

# 폴드별 학습 및 검증
results = []

# 모델 저장 경로 설정
model_save_path = '/content/drive/MyDrive/횡단보도/segmentation/models'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
    print(f'Starting fold {fold}')
    train_subsampler = SubsetRandomSampler(train_ids)
    val_subsampler = SubsetRandomSampler(val_ids)

    train_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=train_subsampler)
    val_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=val_subsampler)

    device =select_device('')
    model = create_deeplabv3plus(num_classes=4).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

In [ ]:
def evaluate_model_on_test(model_path, test_loader, device):
    # 모델 구조 정의 (위에서 정의한 create_deeplabv3plus 함수 사용)
    model = create_deeplabv3plus(num_classes=4).to(device)

    # 저장된 모델 가중치 로드
    model.load_state_dict(torch.load(model_path))

    # 평가 모드 설정
    model.eval()

    # 정확도 계산을 위한 변수 초기화
    total_pixels = 0
    correct_pixels = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            correct_pixels += (predicted == labels).sum().item()
            total_pixels += torch.numel(labels)


    # 정확도 출력
    accuracy = 100 * correct_pixels / total_pixels
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

# 모델 평가
model_path = '/content/drive/MyDrive/횡단보도/4models/model_fold4_epoch72.pth'  # 모델 경로
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
evaluate_model_on_test(model_path, test_loader, device)

Accuracy of the model on the test images: 96.44%


In [ ]:


clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

!python /content/drive/MyDrive/yolov5/train.py --img 640 --batch 16 --epochs 30 --data data.yaml --weights yolov5m.pt --project /content/drive/MyDrive/yolov5/data/res



In [ ]:

# 필요한 라이브러리 설치
!git clone https://github.com/ultralytics/ultralytics
%cd /content/drive/MyDrive/yolov5/
!pip install -r /content/drive/MyDrive/yolov5/requirements.txt # 필요한 라이브러리 설치

import torch
from IPython.display import Image, clear_output  # 결과를 표시하기 위한 라이브러리
!python /content/drive/MyDrive/yolov5/train.py --img 640 --patience 40 --batch 16 --epochs 400 --data /content/drive/MyDrive/yolov5/data/data.yaml --weights yolov5l.pt



2024-02-22 13:16:35.544741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 13:16:35.544791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 13:16:35.546366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5l.pt, cfg=, data=/content/drive/MyDrive/yolov5/data/data.yaml, hyp=drive/MyDrive/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=400, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=drive/MyDrive/yolov5/data/hyps, resume_evolve=None, bucket=, cache=

In [ ]:
!python /content/drive/MyDrive/yolov5/val.py --img 640 --weights /content/drive/MyDrive/yolov5/runs/train/exp11/weights/best.pt --data /content/drive/MyDrive/yolov5/data/data.yaml --task test

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.9/715.9 kB 15.1 MB/s eta 0:00:00
val: data=/content/drive/MyDrive/yolov5/data/data.yaml, weights=['/content/drive/MyDrive/yolov5/runs/train/exp11/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=drive/MyDrive/yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 235.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.3s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v7.0-284-g95ebf68f Python-3.10.12 torch-2.1.0+

In [ ]:
%cat /content/drive/MyDrive/yolov5/data/data.yaml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
cat: /content/drive/MyDrive/yolov5/data/data.yaml: No such file or directory


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # YOLOv5 리포지토리 클론
%cd yolov5

In [ ]:
!pip install PyYAML
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.4/715.4 kB 9.8 MB/s eta 0:00:00


In [ ]:
model = YOLO('yolov8l.pt')

100%|██████████| 83.7M/83.7M [00:00<00:00, 344MB/s]


In [ ]:
# yolov8 학습
model.train(data='/content/drive/MyDrive/yolov5/data/data.yaml', epochs=100, batch= 16, imgsz=640)

Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/yolov5/data/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

100%|██████████| 755k/755k [00:00<00:00, 41.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 90.3MB/s]


AMP: checks passed ✅


...

                   all        120        346      0.857      0.656      0.766      0.424



100 epochs completed in 0.786 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 87.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 87.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.14s/it]


                   all        120        346      0.805      0.702       0.78      0.435
Speed: 0.1ms preprocess, 10.6ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d259af819c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
!python /content/drive/MyDrive/yolov5/val.py --img 640 --weights /content/drive/MyDrive/yolov5/runs/train/exp5/weights/best.pt --task test

val: data=data/coco128.yaml, weights=['/content/drive/MyDrive/yolov5/runs/train/exp5/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-284-g95ebf68f Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs

Dataset not found ⚠️, missing paths ['/content/drive/MyDrive/datasets/coco128/images/train2017']
100% 6.66M/6.66M [00:00<00:00, 145MB/s]
Dataset download success ✅ (4.0s), saved to /content/drive/MyDrive/datasets
100% 755k/755k [00:00<00:00, 111MB/s]
Traceback (most recent call last):
  File "/content/drive/MyDrive/yolov5/val.py", line 430, in <module>
    main(opt)
  File "/content/drive/MyDrive/yolov5/val.py", line 401, i

In [ ]:

import torch
import os
from glob import glob
from PIL import Image
import numpy as np
from torchvision import transforms
!pip install -r /content/drive/MyDrive/yolov5/requirements.txt #경로수정
%cd /content/drive/MyDrive/yolov5
from models.experimental import attempt_load  # YOLOv5 모델 로드 함수
from utils.general import non_max_suppression  # NMS 함수
from utils.torch_utils import select_device  # 디바이스 선택 함수
!pip install segmentation_models_pytorch
import segmentation_models_pytorch as smp #임포트 해야되고

def save_object_detection_results(pred, save_path, img_width, img_height):
    # 이미지 크기를 모델 입력 크기로 설정
    model_input_width, model_input_height = 352, 640
    result_lines = []
    for i, det in enumerate(pred):
        if len(det):
            for *xyxy, conf, cls in reversed(det):
                x_center = ((xyxy[0] + xyxy[2]) / 2) / model_input_width
                y_center = ((xyxy[1] + xyxy[3]) / 2) / model_input_height
                width = (xyxy[2] - xyxy[0]) / model_input_width
                height = (xyxy[3] - xyxy[1]) / model_input_height
                result_line = f"0 {x_center.item()} {y_center.item()} {width.item()} {height.item()}"
                result_lines.append(result_line)

    with open(save_path, 'w') as f:
        f.write('\n'.join(result_lines))

def predict_images_in_folder(image_folder, object_detection_model_path, segmentation_model_path, device, save_dir):
    object_detection_model = attempt_load(object_detection_model_path)  # 객체 탐지 모델 로드
    object_detection_model.to(device).eval()

    segmentation_model = smp.DeepLabV3Plus(  # 도로 영역 분류 모델 로드
        encoder_name="resnet101",
        encoder_weights=None,
        in_channels=3,
        classes=4
    )
    segmentation_model.load_state_dict(torch.load(segmentation_model_path, map_location=device))
    segmentation_model.to(device).eval()

    transform = transforms.Compose([  # 이미지 전처리
        transforms.Resize((640, 352)),
        transforms.ToTensor(),
    ])

    for image_path in glob(os.path.join(image_folder, '*.jpg')):
        image = Image.open(image_path).convert("RGB")
        img_width, img_height = image.size
        img_tensor = transform(image).unsqueeze(0).to(device)

        pred = object_detection_model(img_tensor, augment=False)[0]
        pred = non_max_suppression(pred, 0.3, 0.2, classes=None, agnostic=False)

        object_detection_save_path = os.path.join(save_dir, os.path.basename(image_path).replace('.jpg', '_detection.txt'))
        save_object_detection_results(pred, object_detection_save_path, img_width, img_height)

        seg_pred = segmentation_model(img_tensor)  # 도로 영역 분류 수행
        seg_pred = torch.argmax(seg_pred, dim=1).byte().cpu().numpy()

        road_area_mask = Image.fromarray(seg_pred[0])  # 도로 영역 분류 결과 저장
        road_area_mask_save_path = os.path.join(save_dir, os.path.basename(image_path).replace('.jpg', '_road_area_mask.png'))
        road_area_mask.save(road_area_mask_save_path)

# 예제 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_folder = '/content/drive/MyDrive/yolov5/data/images/test'  # 분석할 이미지 폴더 경로
object_detection_model_path = '/content/drive/MyDrive/yolov5/runs/train/exp5/weights/best.pt'  # 객체 탐지 모델 파일 경로
segmentation_model_path = '/content/drive/MyDrive/횡단보도/segmentation/models/model_fold4_epoch33.pth'  # 도로 영역 분류 모델 파일 경로
save_dir = '/content/drive/MyDrive/yolo_1'  # 결과를 저장할 디렉토리 경로

predict_images_in_folder(image_folder, object_detection_model_path, segmentation_model_path, device, save_dir)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.9/715.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00
/content/drive/MyDrive/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.4 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=a948699839c51b472684df70f4d103ed9b658040ef9e81b31dbdef899e476f6b
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=b9353f328c41

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded


In [ ]:
from PIL import Image, ImageDraw

def visualize_human_labels(human_label_file, output_file):
    # Load human object location labeling file
    with open(human_label_file, 'r') as f:
        human_label_data = f.readlines()

    # Create a white image
    img = Image.new('RGB', (640, 640), color='white')
    draw = ImageDraw.Draw(img)

    # Draw bounding boxes for human objects
    for label in human_label_data:
        label_data = label.strip().split()
        object_center_x, object_center_y, object_width, object_height = map(float, label_data[1:])
        x_min = object_center_x - object_width / 2
        y_min = object_center_y - object_height / 2
        x_max = object_center_x + object_width / 2
        y_max = object_center_y + object_height / 2

        # Draw bounding box of human object
        draw.rectangle([x_min * 640, y_min * 640, x_max * 640, y_max * 640], outline='red', width=2)

    # Save the image
    img.save(output_file)
    print("Human labels visualization saved at", output_file)

def visualize_road_labels(road_label_file, output_file):
    # Load road area detection labeling file
    road_label_img = Image.open(road_label_file)

    # Create a blank image to draw colored road areas
    colored_img = Image.new('RGB', road_label_img.size)
    draw = ImageDraw.Draw(colored_img)

    # Define colors for different road areas
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # Example colors: Red, Green, Blue

    # Highlight road areas with different colors
    for y in range(road_label_img.size[1]):
        for x in range(road_label_img.size[0]):
            pixel_color = road_label_img.getpixel((x, y))
            if pixel_color == (255, 255, 255):  # White color indicates road area
                # Choose a color based on the location or any other criteria
                color_index = (x + y) % len(colors)  # Example: Alternate colors based on x+y coordinate
                draw.point((x, y), fill=colors[color_index])

    # Save the image
    colored_img.save(output_file)
    print("Road labels visualization saved at", output_file)

# Example usage
human_label_file = '/content/240116173846-0043 O.txt'  # Example human object labeling file
#road_label_file = '/content/drive/MyDrive/sejong/test/segment_labels/0001.png'  # Example road area labeling file
output_human_image_file = '/content/1223.png'  # Output visualization image file path for human labels
#output_road_image_file = '/content/drive/MyDrive/sejong/test/001203123/road_labels.png'  # Output visualization image file path for road labels

visualize_human_labels(human_label_file, output_human_image_file)
#visualize_road_labels(road_label_file, output_road_image_file)

Human labels visualization saved at /content/1223.png
